# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [1]:
# off-the-shelf 방식
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

# 변수:값 -> (prompt_template) -prompt-> (model) -응답결과-> (parser) -> 최종결과

In [2]:
from langchain import LLMChain
chain = LLMChain(
    prompt=prompt_template,
    llm=model, 
    output_parser=parser
)
response = chain.invoke({"item":"가방", "count":5})

C:\Users\Playdata\AppData\Local\Temp\ipykernel_10112\1686607166.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:
response

{'item': '가방',
 'count': 5,
 'text': '물론이죠! 가방에 어울리는 멋진 이름 5개를 제안해 드릴게요:\n\n1. **모던 백** (Modern Bag) - 세련된 디자인을 강조하는 이름\n2. **에코 스퀘어** (Eco Square) - 친환경 소재로 만든 가방에 어울리는 이름\n3. **다이나믹 클러치** (Dynamic Clutch) - 활동적인 리듬에 잘 어울리는 클러치백\n4. **로맨틱 토트** (Romantic Tote) - 사랑스러운 분위기를 풍기는 토트백\n5. **유니크 스트랩** (Unique Strap) - 개성 있는 스트랩 디자인을 가진 가방\n\n이 중에서 마음에 드는 이름이 있으면 좋겠네요!'}

In [4]:
print(response["text"])

물론입니다! 가방에 어울리는 이름 5개를 제안해 드릴게요.

1. **에코백** - 자연 친화적인 스타일을 강조한 이름
2. **모던 클러치** - 세련되고 현대적인 느낌을 주는 이름
3. **트래블 메이트** - 여행과 함께하는 가방이라는 의미를 담은 이름
4. **스타일리시 캐리어** - 멋과 실용성을 동시에 표현한 이름
5. **러닝 투고** - 활동적인 라이프스타일에 어울리는 이름

이름이 마음에 드시길 바랍니다!


# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입이나 `함수등 Callable객체`으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassthrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

#### Runnable 예제

In [ ]:
from langchain_core.runnables import Runnable  # 모든 Runnable의 최상위.
# 사용자정의 Runnable
class MyRunnable(Runnable):

    def invoke(self, input_data:str, config:dict=None):
        # invoke(): 구현하는 Runnable이 해야하는 작업을 구현하는 메소드.
        # input_data: 입력값
        # config: 일할 때 필요한 설정값들.
        if config is not None and config.get('lang') == "en":
            return f"Explain {input_data} in one sentences."
        return f"{input_data}에 대해서 한 문장으로 설명해줘."

In [13]:
my_runnable = MyRunnable()
my_runnable.invoke("사과")
my_runnable.invoke("컴퓨터")
my_runnable.invoke("Apple", {"lang":"en"})

'Explain Apple in one sentences.'

In [15]:
from langchain_openai import ChatOpenAI

my_runnable = MyRunnable()
model = ChatOpenAI(model_name="gpt-4o-mini")

prompt = my_runnable.invoke("Apple", {"lang":"en"})
prompt = my_runnable.invoke("Langchain")
response = model.invoke(prompt)
print(response)

content='Langchain은 다양한 언어 모델을 연결하고 활용하여 복잡한 애플리케이션을 구축하는 데 도움을 주는 개발 프레임워크입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 19, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgjIzSM7rZgIo7KuycyA1Kz9XxhNR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--7ff04cc7-a563-40b7-a78a-9a09822de099-0' usage_metadata={'input_tokens': 19, 'output_tokens': 35, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [16]:
# chain -> Runnable | Runnable | Runnable
chain = my_runnable | model
# chain 호출 -> invoke
res = chain.invoke("과일 배")
print(res)

content='과일 배는 달콤하고 즙이 많은 속과 부드러운 껍질을 가진 과일로, 주로 생으로 섭취하거나 요리와 디저트에 사용됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 20, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgjLYzwH8fpTLnnMO1NI5DYLTYKAB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--49b62362-d46d-4ad6-82e9-203286a93874-0' usage_metadata={'input_tokens': 20, 'output_tokens': 45, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
# 기본 체인 구성: prompt_template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

# role: system, user/human, ai/assistant
#       system: 채팅  전체에 적용되는 공통 지침을 지정하는 role
prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 오랜 경력의 한국 관광 가이드입니다. 여행객들에게 설명하듯이 친절하게 답변을 해주세요."),
        ("human", "{query}")
    ]
)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)

guide_chain = prompt_template | model | StrOutputParser()

print(type(guide_chain)) # RunnableSequence: Runnable 타입 
#                           ==> chain도 다른 chain의 구성요소로 포함될 수있다.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [6]:
query = "서울에서 꼭 가봐야되는 여행지를 세 곳만 알려줘."
response = guide_chain.invoke({"query":query})

In [7]:
print(response)

물론입니다! 서울에는 멋진 여행지가 많은데요, 그 중에서도 꼭 가봐야 할 세 곳을 소개해드릴게요.

1. **경복궁**: 서울에서 가장 큰 궁궐인 경복궁은 조선 시대의 왕궁으로, 화려한 건축물과 아름다운 정원이 돋보이는 곳입니다. 특히, 경복궁의 광화문 앞에서 열리는 수문장 교대식은 매일 오전 10시와 오후 2시에 진행되니, 시간을 맞춰서 꼭 관람해 보세요. 경복궁 내에는 국립고궁박물관도 있어 궁과 함께 한국의 역사와 문화를 배울 수 있는 좋은 기회입니다.

2. **명동**: 쇼핑과 맛집으로 가득한 명동은 젊은 여행객들에게 특히 인기가 많아요. 다양한 화장품 가게, 패션 숍들, 그리고 길거리 음식들이 즐비해 있어요. 특히 명동의 길거리 음식으로는 떡볶이, 김밥, 그리고 핫바를 추천드립니다. 즐거운 쇼핑과 맛있는 간식을 경험해보세요!

3. **북촌 한옥마을**: 전통 한국 가옥인 한옥들이 밀집해 있는 북촌 한옥마을은 서울의 고풍스러운 매력을 느낄 수 있는 곳입니다. 좁은 골목길을 거닐며 독특한 건축물들을 감상하거나, 전통 찻집에 들러 한국차를 즐길 수도 있습니다. 특히 사진 찍기 좋은 포토 존이 많아, 인스타그램에 올릴 멋진 사진을 찍기에 좋은 장소입니다.

이 세 곳은 한국의 전통과 현대가 어우러진 서울을 경험할 수 있는 장소이니, 꼭 방문해 보시길 추천드립니다! 즐거운 여행 되세요!


In [23]:
while True:
    query = input("질문:")
    if query == "!quit":
        break
    resp = guide_chain.invoke({"query":query})
    print("User:", query)
    print("AI:", resp)
    print("-"* 50)

User: 경복궁에 대해서 설명해줘.
AI: 안녕하세요! 경복궁에 오신 것을 환영합니다. 경복궁은 대한민국 서울특별시에 위치한 조선 왕조의 대표적인 궁궐 중 하나로, 1395년에 지어졌습니다. 이곳은 조선의 왕들이 살았던 곳으로, 풍부한 역사와 아름다운 건축물로 유명합니다.

경복궁의 이름은 "가득 차고 복이 온다"는 뜻을 가지고 있는데요, 이것은 조선 왕조가 국가의繁栄과 평화를 기원하던 의미를 담고 있습니다. 특히 이 궁궐은 조선 시대의 궁궐 중 가장 크고 화려한 건축물로, 대표적인 건축물로는 근정전, 경회루, 사정전을 들 수 있습니다.

**근정전**은 조선의 왕들이 즉위식이나 중요 의식을 치르던 곳으로, 대규모 행사와 국정을 다루는 장소였습니다. 이곳의 아름다운 외관과 화려한 기둥 위에 얹혀 있는 지붕은 경복궁의 상징이라고 할 수 있습니다.

**경회루**는 궁궐의 연못에 세워진 정자로, 조선 왕들이 연회를 열거나 외로운 저녁을 즐기는 장소였죠. 경회루에 서면 주변의 아름다운 조경과 함께 한 폭의 그림 같은 풍경을 감상할 수 있습니다.

여름철에 인근에서 열리는 전통 공연도 놓치지 말아야 할 관람 포인트입니다. 궁궐 내에서는 매일 정오에 진행되는 교대식도 관람할 수 있으며, 이는 한국의 전통적인 문화를 경험하기 좋은 기회입니다.

경복궁은 단순한 궁궐 이상의 의미를 지니고 있으며, 한국의 역사와 문화가 숨쉬는 장소입니다. 방문하시면 조선 왕조의 과거를 느끼는 특별한 경험이 될 것입니다. 궁궐 내 산책을 즐기시면서 아름다운 경치를 만끽해 보세요!
--------------------------------------------------
User: ?
AI: 안녕하세요! 무엇을 도와드릴까요? 한국 여행에 대해 궁금한 점이나 계획하고 있는 관광지에 대해 알려주시면 친절하게 설명해 드리겠습니다. 편하게 질문해 주세요!
--------------------------------------------------
User: 명동에 가면 꼭 가봐야 할 곳은?
AI: 명

#### RunnableLambda 예제

In [25]:
from langchain_core.runnables import RunnableLambda
# RunnableLambda(함수) -> 함수를 실행하는 Runnable을 생성.
my_runnable2 = RunnableLambda(lambda input_data : f"{input_data}를 한 문장으로 설명해줘.")
my_runnable2.invoke("LLM") 

'LLM를 한 문장으로 설명해줘.'

In [26]:
chain = my_runnable2 | model
chain.invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 방대한 양의 텍스트 데이터를 기반으로 언어를 이해하고 생성할 수 있도록 훈련된 인공지능 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 18, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkAAty6ZVMd6IcUMSqaPIfSyDPhM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f65ccf45-541e-433e-9476-cc9348391ef4-0', usage_metadata={'input_tokens': 18, 'output_tokens': 38, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
def sum(nums:dict):
    return nums[0] + nums[1]

my_runnable2 = RunnableLambda(sum)
my_runnable2.invoke({0:10, 1:20})

# invoke(입력데이터:str|dict, 설정정보:dict)
## 입력데이터가 여러개일 경우 dict등의 자료구조를 이용해서 받는다.

30

#### RunnablePassthrough 예제

In [10]:
# 1 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달.
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")
RunnablePassthrough().invoke({"Key":"value"})
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [32]:
# 2. 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 전달.
#   -> 입력으로 dictionary 받아서 거기에 item을 추가.
# RunnablePassthrough.assign(key1=Runnable, key2=Runnable, ...)
#  - 받은 dictionary에 "key1":Runnable반환값, "key2":Runnable반환값, .. 추가해서 다음으로 전달.
address_runnable = RunnableLambda(lambda x: "서울시 금천구")  #"서울시 금천구" 를 반환.
phone_runnable = RunnableLambda(lambda x: "010-1111-2222")

RunnablePassthrough.assign(address=address_runnable, phone=phone_runnable).invoke({"name":"홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

#### RunnableSequence 예제

In [ ]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)

chain = run1 | run2
print(type(chain))
chain.invoke(30)


<class 'langchain_core.runnables.base.RunnableSequence'>


62

In [35]:
chain2 = RunnableSequence(run1, run2)  #(prompt_template, model,  output_parser)
chain2.invoke(100)

202

#### RunnableParallel 예제

In [14]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)
run3 = RunnableLambda(lambda x: x // 3)

runnable = RunnableParallel(
    {
        "result1":run1,
        "result2":run2,
        "result3":run3,
        "result4":RunnablePassthrough()  # 앞에서 받은 값을 그대로 다음에 전달.
    }
)
# Runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary를 반환.
runnable.invoke(20)

{'result1': 21, 'result2': 40, 'result3': 6, 'result4': 20}

In [17]:
# LCEL 안에서는 RunnableParallel은 { } 로 표현할 수있다.
run0 = RunnableLambda(lambda x: x + 100)
# run0 -> {run1, run2, run3} -> ..
chain = run0 | {"result1":run1,
                "result2":run2,
                "result3":run3}

chain.invoke(20)

{'result1': 121, 'result2': 240, 'result3': 40}

#### LCEL Chain 예제

In [18]:
# 음식 이름을 받아서 레시피를  출력하는 chain을 구성
# prompt template -> model -> output parser 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent
prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 숙련된 요리 연구가입니다. 요청한 음식의 레시피를 작성해 주세요.
                    
    # Input data
    음식이름: {food}
    """)
)
model = ChatOpenAI(model_name="gpt-4o-mini")
food_chain = prompt_template | model | StrOutputParser()


In [19]:
response = food_chain.invoke({"food":"pasta"})

In [21]:
print(response)

### 파스타 레시피

파스타는 여러 가지 소스와 재료로 다양하게 즐길 수 있는 이탈리아의 대표적인 요리입니다. 여기 기본적인 파스타 레시피를 제공합니다.

#### 재료
- 파스타 (스파게티, 펜네 등) 200g
- 물 2L
- 소금 1 큰술
- 올리브 오일 2 큰술
- 마늘 3쪽 (다진 것)
- 방울토마토 200g (반으로 자른 것)
- 바질 잎 한 줌 (신선한 것 또는 말린 것)
- 소금, 후추 (맛 조절용)
- 파마산 치즈 (선택사항)

#### 조리 도구
- 큰 냄비
- 팬
- 나무 숟가락
- 체반

#### 조리 방법

1. **파스타 삶기**
   - 큰 냄비에 물 2L를 붓고 소금을 넣은 후 끓입니다.
   - 물이 끓으면 파스타를 넣고 포장지에 적힌 시간에 맞춰 알 덴테 상태로 삶아줍니다.
   - 삶은 후 체반에 걸러 물기를 빼고, 약간의 올리브 오일을 넣어 주어 서로 달라붙지 않게 해줍니다.

2. **소스 만들기**
   - 팬에 올리브 오일을 두르고 중간 불에서 다진 마늘을 넣고 볶습니다. 마늘의 향이 올라오기 시작하면 방울토마토를 추가합니다.
   - 방울토마토가 흐물흐물해질 때까지 5-7분 정도 볶습니다.
   - 소금과 후추로 간을 합니다.

3. **파스타 섞기**
   - 삶은 파스타를 팬에 넣고, 소스와 잘 섞어줍니다.
   - 마지막으로 바질 잎을 넣고 다시 한번 섞어 줍니다.

4. **플레이팅**
   - 접시에 파스타를 담고, 원한다면 파마산 치즈를 갈아서 뿌려줍니다.
   - 추가로 바질 잎을 올려 장식하면 더욱 아름답습니다.

#### 팁
- 다양한 재료를 추가하여 더욱 풍성한 맛을 낼 수 있습니다. 예를 들어, 새우, 베이컨, 채소 등을 추가해보세요.
- 파스타를 삶을 때 넉넉한 소금을 사용하면 맛이 더욱 살아납니다.

맛있게 요리하세요!


In [22]:
# 번역 chain -> 입력된 내용을 지정한(입력한) 언어로 번역하는 체인.
# prompt template -> model -> output parser 

prompt_template_trans = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 다국어가능한 숙력된 번역가입니다. 
    요청된 문장을 {language} 로 번역해 주세요.
                    
    # Input data(번역할 문장)
    {content}
                    
    # Output Indicator
    - 번역한 내용만 출력해 주세요.
    """)
)

translate_chain = prompt_template_trans | model | StrOutputParser()


In [23]:
translate_chain.invoke({"content":"안녕하세요", "language":"독일어"})
# 안녕하세요->독일어로 번역한 응답.



'Hallo'

In [55]:
response_kor = translate_chain.invoke({"content":response, "language":"한국어"})

In [56]:
print(response_kor)

물론입니다! 클래식한 파스타 요리인 스파게티 알리오 올리오(마늘과 올리브 오일을 곁들인 스파게티)의 간단한 레시피를 소개합니다.

### 스파게티 알리오 올리오 레시피

#### 재료:

- 스파게티 400g
- 마늘 6쪽, 얇게 슬라이스
- 엑스트라 버진 올리브 오일 1/2컵
- 붉은 고추 flakes 1/4티스푼 (입맛에 맞게 조절)
- 소금 (맛에 따라)
- 갓 갈은 후추 (맛에 따라)
- 신선한 파슬리 1/4컵, 다진 것
- 파르마산 치즈 (선택 사항, 서빙용)
- 레몬 제스트 (선택 사항, 서빙용)

#### 조리법:

1. **파스타 삶기**:
   - 큰 냄비에 소금을 넣은 물을 끓입니다. 스파게티를 추가하고 패키지 지침에 따라 알던테가 될 때까지 삶습니다. 약 1컵의 파스타 물을 남겨두고 파스타를 Drain합니다.

2. **마늘과 오일 준비하기**:
   - 파스타를 삶는 동안 큰 팬에 중불에서 올리브 오일을 가열합니다.
   - 슬라이스한 마늘과 붉은 고추 flakes를 넣습니다. 마늘이 황금빛 갈색이 되고 향이 날 때까지 자주 저어가며 3-4분간 요리합니다. 마늘을 태우지 않도록 주의하세요. 태우면 쓴맛이 날 수 있습니다.

3. **파스타와 소스 결합**:
   - Drain한 스파게티를 마늘과 오일이 들어 있는 팬에 추가합니다. 파스타가 고르게 오일에 묻도록 잘 섞어줍니다.
   - 만약 파스타가 건조해 보인다면, 남겨두었던 파스타 물을 조금 추가합니다(1/4컵으로 시작하고 필요에 따라 추가). 원하는 농도가 될 때까지 조절하세요.

4. **간 맞추기**:
   - 소금과 갓 갈은 후추로 간을 맞춥니다. 다진 파슬리를 넣고 잘 섞습니다.

5. **서빙**:
   - 파스타를 접시에 담고 원한다면 추가 파슬리, 간 파르마산 치즈, 레몬 제스트로 장식합니다. 즉시 서빙하세요.

맛있게 만든 스파게티 알리오 올리오를 즐기세요!


In [59]:
translate_chain.invoke({"content":response, "language":"일본어"})

'もちろん！クラシックなパスタ料理のシンプルなレシピを紹介します：アーリオ・エ・オリオのスパゲッティ（にんにくとオリーブオイルのスパゲッティ）。\n\n### アーリオ・エ・オリオのスパゲッティレシピ\n\n#### 材料：\n\n- スパゲッティ 400g\n- にんにく 6片（薄切り）\n- エキストラバージンオリーブオイル 1/2カップ\n- 赤唐辛子フレーク 1/4小さじ（お好みで調整）\n- 塩（お好みで）\n- 黒コショウ（お好みで）\n- 新鮮なパセリ 1/4カップ（刻んだもの）\n- パルメザンチーズ（お好みで、提供時に）\n- レモンの皮（お好みで、提供時に）\n\n#### 作り方：\n\n1. **パスタを茹でる**：\n   - 大きな鍋に塩を加えた水を沸騰させます。スパゲッティを加え、パッケージの指示に従ってアルデンテになるまで茹でます。パスタの水を約1カップ取っておき、パスタを水切りします。\n\n2. **にんにくとオイルの準備**：\n   - パスタが茹でている間、大きなフライパンに中火でオリーブオイルを熱します。\n   - 薄切りにしたにんにくと赤唐辛子フレークを加え、にんにくが黄金色になり香りが立つまで、頻繁にかき混ぜながら約3～4分料理します。にんにくを焦がさないように注意してください。焦げると苦味が出ます。\n\n3. **パスタとソースを合わせる**：\n   - 水切りしたスパゲッティをにんにくとオイルのフライパンに加えます。パスタをオイルで均等にコーティングするように混ぜます。\n   - パスタが乾いているようであれば、取っておいたパスタの水を少しずつ加え（最初は1/4カップから始め、必要に応じて追加）、希望の濃度に調整します。\n\n4. **味付け**：\n   - 塩と新鮮に挽いた黒コショウで味を調えます。刻んだパセリを加え、よく混ぜます。\n\n5. **盛り付け**：\n   - パスタを皿に盛り、追加のパセリ、すりおろしたパルメザンチーズ、必要であればレモンの皮で飾ります。すぐに提供します。\n\nおいしい手作りのアーリオ・エ・オリオスパゲッティをお楽しみください！'

## Chain과 Chain간의 연결

In [24]:
# food_chain ----> translate_chain
## food_chain_prompt: 변수 - food
## translate_chain : 변수 - language, content

# RunnableParallel({"key":Runnable, "key2":Runnable})
# LCEL에서 RunnableParallel => {"key":Runnable, "key2":Runnable} |
chain = {"content":food_chain, 
         "language":RunnableLambda(lambda x : x["language"])} | translate_chain

In [ ]:
# chain = {"content":food_chain, 
#          "language":RunnableLambda(lambda x : x["language"]) } | RunnablePassthrough()
# chain.invoke({"food":"파스타", "language":"영어"})

{'content': '## 파스타 레시피\n\n### 재료\n- 스파게티 면 200g\n- 올리브유 2큰술\n- 마늘 3쪽 (다진 것)\n- 양파 1개 (다진 것)\n- 방울토마토 200g (반으로 자른 것)\n- 시금치 100g\n- 바질 잎 적당량 (옵션)\n- 소금과 후추 (간 맞추기)\n- 파마산 치즈 (갈아서, 옵션)\n\n### 조리 도구\n- 큰 냄비\n- 팬\n- 나무 주걱\n- 체반\n\n### 조리 방법\n\n1. **면 삶기**\n   - 큰 냄비에 물을 끓입니다. 물이 끓으면 소금을 넣고 스파게티 면을 넣습니다. 포장지에 적힌 시간 동안 삶아주세요(보통 8-10분).\n   - 면이 다 삶아지면 체반에 걸러 물기를 제거하고, 약간의 올리브유를 뿌려 고루 섞어줍니다.\n\n2. **소스 준비**\n   - 팬에 올리브유를 두르고 중불에서 열을 가합니다. 다진 마늘을 넣고 향이 나기 시작할 때까지 약 1분간 볶습니다.\n   - 다진 양파를 추가하고, 양파가 투명해질 때까지 볶습니다.\n\n3. **토마토와 시금치 추가**\n   - 반으로 자른 방울토마토를 팬에 추가하고, 3-4분간 요리하여 토마토가 부드러워질 때까지 조리합니다.\n   - 시금치를 넣고 시금치가 시들해질 때까지 젓습니다.\n\n4. **면과 혼합**\n   - 삶은 스파게티 면을 팬에 넣고, 전체 재료와 잘 섞이도록 저어줍니다. 필요에 따라 소금과 후추로 간을 맞춥니다.\n   - 팬에서 불을 끈 후, 신선한 바질 잎을 위에 올려줍니다.\n\n5. **서빙**\n   - 완성된 파스타를 접시에 담고, 원한다면 갈아놓은 파마산 치즈를 뿌려 서빙합니다.\n\n### 팁\n- 다양한 재료 (예: 새우, 베이컨, 크림 소스 등)를 추가하여 맛을 변형할 수 있습니다.\n- 파스타는 항상 즉석에서 조리한 후 바로 서빙하는 것이 가장 맛있습니다. \n\n맛있게 드세요!',
 'language': '영어'}

In [25]:
res = chain.invoke({"food":"파스타", "language":"영어"})
# food -> food_chain, language -> translate_chain
# food_chain 최종결과(레시피) => {"content":레시피} -전달-> translate_chain


In [26]:
print(res)

## Pasta Recipe

### Ingredients
- 200g spaghetti
- 2 tablespoons olive oil
- 2 cloves garlic (minced)
- 1 onion (minced)
- 200g cherry tomatoes (halved)
- A handful of basil leaves (fresh, chopped)
- Parmesan cheese (grated, to taste)
- Salt and pepper (to taste)

### Cooking Time
- Preparation time: 10 minutes
- Cooking time: 15 minutes
- Total time: 25 minutes

### Cooking Method
1. **Cook the Pasta**: Boil water in a large pot and add salt. When the water boils, add the spaghetti and cook according to the time indicated on the package. Cook until al dente. Once cooked, drain the pasta and toss it with a bit of olive oil to prevent sticking.

2. **Make the Sauce**: Heat olive oil in a pan over medium-low heat and add minced garlic and onion. Sauté until the onion becomes translucent.

3. **Add Tomatoes**: Place the chopped cherry tomatoes in the pan and sauté for another 3-5 minutes until the tomatoes soften and release their juices. Season with salt and pepper.

4. **Combine Pasta 

In [79]:
chain.invoke({"food":"냉면", "language":"몽골어"})

'### Халуун ногооны жор\n\n#### Орц\n* Халуун ногооны будаа (булгасны будаа эсвэл гурилан будаа) 200г\n* Шөлийг (үхрийн шөл эсвэл загасны шөл) 1L\n* Сүү (2 том халбага)\n* Цуу (2 том халбага)\n* Элсэн чихэр (1 том халбага)\n* Үхрийн саримс (1 жижиг халбага)\n* Огурцы (1 ш)\n* Нэрс (эсвэл алим) (1/2 ш)\n* Төмс (2 ш)\n* Нори (цамцлаг)\n* Улаан перец (сонголт)\n* Сансар (сонголт)\n\n#### Шөл бэлтгэх\n1. **Шөл хийх:** Үхрийн 200г болон 1L ус тавагтаа хийж буцалгана. Буцалж эхлэхэд дунд гал дээр 30 минут орчим буцалгана.\n2. **Шөлний амт тохируулах:** Шөл ууж идэж, амт нь гүнзгий байх үед 2 том халбага сүү, 1 жижиг халбага үхрийн саримсыг нэмж дахин буцалгаад, галын тагийг хааж хөргөнө.\n\n#### Будаа бэлтгэх\n1. **Будаа буцалгах:** Тавагт ус буцалгаж, халуун ногооны будааг хийнэ. Савны зааварт заасан буцалгах хугацааг анхаарч үзүүлэхэд тохиромжтой хугацаагаар буцална.\n2. **Будаагаа хүйтэн усаар угаах:** Будаа буцалсны дараа хүйтэн усанд хэд хэдэн удаа угааж, гадаргуу нь жигд чипс мэт байх 

# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

In [29]:
from langchain_core.runnables import RunnableLambda
# 기존에 정의된 함수. 이 함수를 Runnable로 정의
def plus(num1, num2, num3):
    return num1 + num2 + num3

def wrapper_plus(nums:list):
    return plus(nums[0], nums[1], nums[2])

run1 = RunnableLambda(wrapper_plus)
# run1.invoke(1, 2, 3)
run1.invoke([1, 2, 3])
# invoke(input_data:첫번째, config:RunnableConfig-설정정보=None)

6

In [3]:
# 소재를 이용해서 이야기를 생성 chain
# 장문을 입력 받아서 요약하는 chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

True

In [16]:
story_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)
story_prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction 
    당신은 아이들을 위한 이야기를 창작하는 스토리텔러입니다.
    주어진 소재로 잠자리에서 아이들에게 들려줄 재미있는 이야기를 만들어 주세요.
                    
    # Input Data
    소재: {topic}
                    
    # Output Indicator
    - 이야기는 30문장 이내로 구성해 주세요.
    - 이야기를 구어체로 작성해 주세요.
    """)
)
story_chain = story_prompt_template | story_model | StrOutputParser()

In [17]:
# 요약 chain
summary_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
summary_prompt = PromptTemplate(
    template=dedent("""
    # Instruction 
    주어진 문장 내용을 2문장으로 요약 해주세요.
                    
    # Input Data
    {content}
    """)
)
summary_chain = summary_prompt | summary_model | StrOutputParser()

In [6]:
chain = story_chain | summary_chain
chain.invoke({"topic":"호랑이"})

'호호라는 호랑이는 사람들과 친해지고 싶어하며 숲 속의 다양한 동물들과 친구가 되려고 노력했습니다. 결국 모든 동물들이 호호를 좋아하게 되었고, 숲은 평화롭고 즐거운 곳이 되었다는 이야기입니다.'

In [18]:
from langchain_core.runnables import chain

# 사용자 정의 체인 -> 흐름을 원하는 구조로 정의할 수있다.
@chain
def custom_chain(topic:str)->dict[str, str]:
    # story_chain과 summary_chain을 원하는 흐름으로 구현.
    # story_chain의 결과와 summary_chain의 결과를 모두 반환하도록 처리.
    story = story_chain.invoke({"topic":topic})
    if True:
        summary = summary_chain.invoke({"content":story})

    return {"전체이야기":story, "이야기요약":summary}

In [9]:
type(custom_chain)

langchain_core.runnables.base.RunnableLambda

In [19]:
res = custom_chain.invoke("호랑이")

In [12]:
print(type(res))
print(res.keys())

<class 'dict'>
dict_keys(['전체이야기', '이야기요약'])


In [20]:
print(res['전체이야기'])

옛날 옛적, 깊은 숲속에 호랑이 한 마리가 살고 있었어요. 이 호랑이는 다른 호랑이들과는 좀 달랐어요. 이름은 '호롱이'였고, 호롱이는 매우 호기심이 많았죠.

어느 날, 호롱이는 숲 속에서 반짝이는 것을 발견했어요. "우와! 저건 뭐지?" 하고 궁금해졌죠. 그래서 호롱이는 그 반짝이는 걸 보러 다가갔어요. 가까이 가보니, 그것은 예쁜 보석이었어요!

"이건 꼭 친구들에게 보여줘야 해!" 호롱이는 생각했죠. 그래서 친구들에게 달려가면서 큰 소리로叫렸어요. "얘들아, 와서 봐! 신기한 보석을 찾았어!"

호랑이 친구들, 곰 친구들, 그리고 새 친구들이 호롱이를 따라갔어요. 숲 속을 달리며 모두 신나게 이야기했죠. "와우! 호롱이, 진짜로 보석이야?" 친구들이 신기해하자, 호롱이는 자랑스럽게 말했어요. "오늘의 주인공은 나야!"

하지만 돌고 돌아 결국 도착했을 때, 호롱이는 보석이 보이지 않았어요. "어? 어디 갔지?" 친구들은 각자 바닥을 샅샅이 뒤졌어요. 그런데 갑자기, 숲이 멈춘 듯 조용해졌어요.

그때, 속삭이는 소리가 들렸어요. "여기서 뭘 찾아요?" 나무 아래에서 나타난 건 바로 숲의 수호령인 강아지였어요. 호롱이는 당황해서 말했어요. "아, 우리는 반짝이는 보석을 찾고 있었어요."

강아지가 미소를 지으며 말했어요. "보석은 겉모습이 전부가 아니야. 진정한 보석은 친구와 함께하는 가치란다." 호롱이는 그 말을 듣고 깜짝 놀랐어요. "그래! 친구와의 시간도 반짝이는 보석 같은 거구나!"

친구들은 서로의 소중함을 느끼며 함께 즐거운 시간을 보냈어요. 이제 호롱이는 보석이 필요 없었어요. 친구들과의 모험이 그에게 가장 큰 보물이었으니까요.

그날 이후, 호롱이는 늘 친구들과 함께하는 법을 배웠어요. 숲의 동물들은 언제나 함께 놀고, 서로를 돕는 소중한 친구가 되었답니다. 그리고 호롱이는 그 날의 이야기를 잊지 않았어요. "가장 귀한 보석은 친구야!" 

아이들아, 오늘 호롱이처럼 친구들과의 소중한 순간을 잘 간직하렴!


In [21]:
print(res['이야기요약'])

호롱이는 숲에서 반짝이는 보석을 발견하고 친구들에게 자랑하려 했지만, 결국 보석을 잃어버렸다. 그러나 숲의 수호령 강아지의 말로 친구와의 소중한 시간이 진정한 보석임을 깨닫고, 이후로 친구들과 함께하는 법을 배웠다.


# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [1]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache

# 대화내역을 cache에 저장 시작
# set_llm_cache(InMemoryCache())  # 메모리에 저장

set_llm_cache(SQLiteCache("llm_cache.sqlite"))

model = ChatOpenAI(model_name="gpt-4o-mini")
response = model.invoke("주요 프로그래밍 언어 5개를 소개해줘.")

In [2]:
print(response.content)

다양한 프로그래밍 언어가 있지만, 다음 다섯 가지는 특히 널리 사용되고 중요한 언어들입니다.

1. **파이썬 (Python)**:
   - 사용 용도: 웹 개발, 데이터 분석, 인공지능, 과학 계산 등.
   - 특징: 간결하고 읽기 쉬운 문법으로, 초보자와 전문가 모두에게 인기가 높습니다. 다양한 라이브러리와 프레임워크 지원으로 많은 분야에서 활용됩니다.

2. **자바스크립트 (JavaScript)**:
   - 사용 용도: 웹 개발 (프론트엔드 및 백엔드), 모바일 앱 개발 등.
   - 특징: 웹 브라우저에서 실행되는 스크립트 언어로, 동적인 웹 페이지를 만들 수 있습니다. Node.js와 같은 런타임을 통해 서버측 프로그래밍에도 사용됩니다.

3. **자바 (Java)**:
   - 사용 용도: 엔터프라이즈 애플리케이션, 모바일 앱 (안드로이드), 웹 애플리케이션 등.
   - 특징: 플랫폼 독립적이며, "한 번 작성하면 어디서나 실행된다"(WORA)는 원칙을 따릅니다. 강력한 객체 지향 언어입니다.

4. **C# (C 샤프)**:
   - 사용 용도: 웹 애플리케이션, 게임 개발 (Unity), 데스크탑 애플리케이션 등.
   - 특징: 마이크로소프트에서 개발한 언어로, .NET 프레임워크와 함께 사용됩니다. 객체 지향 및 함수형 프로그래밍을 지원합니다.

5. **C++**:
   - 사용 용도: 시스템 소프트웨어, 게임 개발, 성능이 중요한 애플리케이션 등.
   - 특징: C 언어의 확장으로, 객체 지향 프로그래밍을 지원합니다. 높은 성능과 메모리 관리를 좀 더 섬세하게 할 수 있어 다양한 분야에서 활용됩니다.

각 언어는 고유의 특성과 용도가 있으며, 프로젝트의 요구에 따라 적합한 언어를 선택하는 것이 중요합니다.


In [3]:
response = model.invoke("주요 프로그래밍 언어 5개를 소개해줘.")
print(response.content)

다양한 프로그래밍 언어가 있지만, 다음 다섯 가지는 특히 널리 사용되고 중요한 언어들입니다.

1. **파이썬 (Python)**:
   - 사용 용도: 웹 개발, 데이터 분석, 인공지능, 과학 계산 등.
   - 특징: 간결하고 읽기 쉬운 문법으로, 초보자와 전문가 모두에게 인기가 높습니다. 다양한 라이브러리와 프레임워크 지원으로 많은 분야에서 활용됩니다.

2. **자바스크립트 (JavaScript)**:
   - 사용 용도: 웹 개발 (프론트엔드 및 백엔드), 모바일 앱 개발 등.
   - 특징: 웹 브라우저에서 실행되는 스크립트 언어로, 동적인 웹 페이지를 만들 수 있습니다. Node.js와 같은 런타임을 통해 서버측 프로그래밍에도 사용됩니다.

3. **자바 (Java)**:
   - 사용 용도: 엔터프라이즈 애플리케이션, 모바일 앱 (안드로이드), 웹 애플리케이션 등.
   - 특징: 플랫폼 독립적이며, "한 번 작성하면 어디서나 실행된다"(WORA)는 원칙을 따릅니다. 강력한 객체 지향 언어입니다.

4. **C# (C 샤프)**:
   - 사용 용도: 웹 애플리케이션, 게임 개발 (Unity), 데스크탑 애플리케이션 등.
   - 특징: 마이크로소프트에서 개발한 언어로, .NET 프레임워크와 함께 사용됩니다. 객체 지향 및 함수형 프로그래밍을 지원합니다.

5. **C++**:
   - 사용 용도: 시스템 소프트웨어, 게임 개발, 성능이 중요한 애플리케이션 등.
   - 특징: C 언어의 확장으로, 객체 지향 프로그래밍을 지원합니다. 높은 성능과 메모리 관리를 좀 더 섬세하게 할 수 있어 다양한 분야에서 활용됩니다.

각 언어는 고유의 특성과 용도가 있으며, 프로젝트의 요구에 따라 적합한 언어를 선택하는 것이 중요합니다.


In [2]:
model.invoke("Langchain에 대해서 소개해줘.")

AIMessage(content='Langchain은 자연어 처리(NLP)와 여러 AI 기술을 통합하여 대화를 생성하고 관리하는 데 도움을 주는 프레임워크입니다. 이 프레임워크는 특히 언어 모델을 활용하여 다양한 언어 관련 작업을 수행할 수 있도록 설계되었습니다.\n\nLangchain의 주요 특징은 다음과 같습니다:\n\n1. **모듈화**: Langchain은 구성 요소를 모듈화하여 각 요소를 독립적으로 사용할 수 있게 설계되어 있습니다. 이를 통해 개발자는 특정 요구사항에 맞게 시스템을 조정하고 확장할 수 있습니다.\n\n2. **언어 모델 통합**: 여러 AI 언어 모델을 쉽게 통합할 수 있으며, 이를 통해 다양한 종류의 질문 응답, 텍스트 생성, 대화형 인터페이스 등을 구현할 수 있습니다.\n\n3. **데이터 연결**: Langchain은 데이터베이스, API, 파일 시스템과 같은 다양한 데이터 소스와 쉽게 연결할 수 있어, 필요한 데이터를 효율적으로 조회하고 사용할 수 있습니다.\n\n4. **유연성**: 사용자가 원하는 방식으로 대화 흐름을 커스터마이즈하고, 상황에 맞는 대화 응답을 만들 수 있는 유연성을 제공합니다.\n\n5. **사용 사례**: 고객 지원 챗봇, 개인 비서, 교육용 도구 등 다양한 응용 프로그램에 활용될 수 있습니다.\n\nLangchain은 개발자들이 빠르게 언어 모델을 기반으로 한 애플리케이션을 구축할 수 있도록 도와주는 강력한 도구로, 점점 더 많은 분야에서 그 활용도가 증가하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 341, 'prompt_tokens': 15, 'total_tokens': 356, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0

In [5]:
model.invoke("Langchain에 대해서 소개해줘.")

AIMessage(content='Langchain은 자연어 처리(NLP)와 여러 AI 기술을 통합하여 대화를 생성하고 관리하는 데 도움을 주는 프레임워크입니다. 이 프레임워크는 특히 언어 모델을 활용하여 다양한 언어 관련 작업을 수행할 수 있도록 설계되었습니다.\n\nLangchain의 주요 특징은 다음과 같습니다:\n\n1. **모듈화**: Langchain은 구성 요소를 모듈화하여 각 요소를 독립적으로 사용할 수 있게 설계되어 있습니다. 이를 통해 개발자는 특정 요구사항에 맞게 시스템을 조정하고 확장할 수 있습니다.\n\n2. **언어 모델 통합**: 여러 AI 언어 모델을 쉽게 통합할 수 있으며, 이를 통해 다양한 종류의 질문 응답, 텍스트 생성, 대화형 인터페이스 등을 구현할 수 있습니다.\n\n3. **데이터 연결**: Langchain은 데이터베이스, API, 파일 시스템과 같은 다양한 데이터 소스와 쉽게 연결할 수 있어, 필요한 데이터를 효율적으로 조회하고 사용할 수 있습니다.\n\n4. **유연성**: 사용자가 원하는 방식으로 대화 흐름을 커스터마이즈하고, 상황에 맞는 대화 응답을 만들 수 있는 유연성을 제공합니다.\n\n5. **사용 사례**: 고객 지원 챗봇, 개인 비서, 교육용 도구 등 다양한 응용 프로그램에 활용될 수 있습니다.\n\nLangchain은 개발자들이 빠르게 언어 모델을 기반으로 한 애플리케이션을 구축할 수 있도록 도와주는 강력한 도구로, 점점 더 많은 분야에서 그 활용도가 증가하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 341, 'prompt_tokens': 15, 'total_tokens': 356, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0

In [3]:
model.invoke("Langchain에 대해서 설명해줘.")

AIMessage(content='Langchain은 다양한 언어 모델과 애플리케이션을 연결하기 위한 프레임워크입니다. 이 프레임워크는 언어 모델을 활용하여 자연어 처리 애플리케이션을 쉽게 구축하고 배포할 수 있도록 돕습니다. Langchain은 여러 모듈로 구성되어 있어, 다양한 작업을 수행할 수 있습니다. 주요 특징은 다음과 같습니다:\n\n1. **프롬프트 관리**: Langchain은 사용자가 입력하는 프롬프트를 효율적으로 관리하고 조작할 수 있도록 돕습니다. 이를 통해 언어 모델의 응답 품질을 향상시킬 수 있습니다.\n\n2. **체인과 파이프라인**: Langchain은 여러 작업을 연결하여 체인을 형성할 수 있습니다. 예를 들어, 사용자의 질문에 대한 응답을 생성한 후, 그 응답을 기반으로 추가 정보를 검색하는 과정 등을 자동화할 수 있습니다.\n\n3. **메모리**: Langchain은 대화가 진행되는 동안 기억을 통해 이전 대화 내용을 활용할 수 있는 기능을 제공합니다. 이를 통해 보다 맥락에 맞는 응답을 생성할 수 있습니다.\n\n4. **데이터 소스 통합**: Langchain은 다양한 데이터 소스와 통합할 수 있는 기능을 제공하여, 언어 모델이 실시간으로 정보를 검색하거나 처리할 수 있도록 합니다.\n\n5. **다양한 언어 모델 지원**: Langchain은 OpenAI, Hugging Face 등 다양한 언어 모델과 호환되어 유연하게 사용할 수 있습니다.\n\n이러한 특징들 덕분에 Langchain은 챗봇, 자동화된 고객 지원 시스템, 콘텐츠 생성 등 다양한 애플리케이션에 활용될 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 368, 'prompt_tokens': 15, 'total_tokens': 383, 'completion_tokens_details': {'accepted_prediction_

# Streaming 방식 응답처리
- LLM 응답의 완료를 기다리지 않고 실시간으로 생성되는 토큰/청크를 받아서 처리한다.
- `모델.invoke(input, config)->응답데이터`: 모델의 응답을 기다렸다가 완료되면 한번에 반환.
- `모델.stream(input, config)->Iterator`: 모델이 토큰을 생성하면 바로 바로 반환.

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini")
res = model.stream("AI에 대해서 설명해줘.")
print(type(res))

<class 'generator'>


In [10]:
for token in model.stream("AI에 대해서 설명해줘."):
    print(token.content, end="")

인공지능(AI, Artificial Intelligence)은 기계나 컴퓨터가 인간과 유사한 방식으로 지능을 발휘할 수 있도록 만드는 기술 및 이론을 말합니다. AI는 다양한 분야에서 활용되며, 주로 다음과 같은 두 가지 주요 분야로 나눌 수 있습니다.

1. **약한 인공지능(Weak AI)**: 특정 작업이나 문제를 해결하기 위해 설계된 AI로, 예를 들어 음성 인식, 이미지 처리, 추천 시스템 등이 있습니다. 현재 사용되는 대부분의 AI 시스템은 약한 인공지능에 해당합니다.

2. **강한 인공지능(Strong AI)**: 인간의 지능과 유사한 수준의 사고력과 이해력을 가진 AI를 의미합니다. 이는 아직 개발되지 않았으며, 이론적으로나마 모든 종류의 문제를 스스로 이해하고 해결할 수 있는 능력을 가진 AI를 지칭합니다.

AI 기술은 머신러닝(기계 학습), 딥러닝(심층 학습), 자연어 처리(NLP) 등 여러 가지 하위 분야로 나뉘어 발전하고 있습니다.

- **머신러닝**: 데이터에서 패턴을 학습하여 예측이나 결정을 내릴 수 있도록 하는 기술입니다. 알고리즘이 데이터에서 학습하여 성능을 개선하는 방식입니다.

- **딥러닝**: 신경망을 기반으로 한 머신러닝의 한 분야로, 다층 구조를 통해 복잡한 데이터 패턴을 인식할 수 있습니다. 이미지 인식과 자연어 처리 분야에서 큰 성과를 보이고 있습니다.

- **자연어 처리(NLP)**: 컴퓨터가 인간의 언어를 이해하고 생성할 수 있도록 하는 기술로, 번역기, 챗봇, 음성 인식 시스템 등에 사용됩니다.

AI의 발전은 다양한 산업에 혁신을 가져오고 있으며, 의료, 금융, 제조, 서비스업 등 여러 분야에서 효율성을 높이고 새로운 가능성을 열어주고 있습니다. 하지만 AI의 발전과 그로 인한 사회적, 윤리적 문제에 대한 논의도 중요해지고 있습니다.